# DLMDSPWP01

## Pre-Requesites
The next cell loads all of the requirements needed for this notebook. For a setup guide (for MacOSX) please consult the [README](./README.md)

If no result is ready yet, please execute the cells in [calculate.ipynb](./calculate.ipynb) first!

In [13]:
import pandas as pd
import bokeh
from bokeh.plotting import figure, show
from bokeh.layouts import row

from services.sql_database import SQLDatabase

In [14]:
bokeh.io.reset_output()
bokeh.io.output_notebook()

Loading BokehJS ...

In [15]:
sql_database = SQLDatabase()

In [16]:
with sql_database.engine.connect() as connection:
    train_df = pd.read_sql_table(table_name="train", con=connection)
    ideal_df = pd.read_sql_table(table_name="ideal", con=connection)
    test_df = pd.read_sql_table(table_name="test", con=connection)

## Train Plots

In [17]:
train_dp = [[]]
cols_per_row = 2
for i in range(4):
    if len(train_dp[-1]) >= cols_per_row:
        train_dp.append([])
    
    fig = figure(width=500, height=500, background_fill_color="#FFFFFF")
    fig.circle(
        train_df["X"].to_list(), 
        train_df[f"Y{i+1}"].to_list(), 
        size=5, 
        color="#123456", 
        alpha=0.8
    )

    train_dp[-1].append(fig)

for item in train_dp: 
    show(row(*item), notebook_handle=True)

## Ideal Plots

In [18]:
ideal_dp = [[]]
cols_per_row = 4
for i in range(50):
    if len(ideal_dp[-1]) >= cols_per_row:
        ideal_dp.append([])
    
    fig = figure(width=250, height=250, background_fill_color="#FFFFFF")
    fig.circle(
        ideal_df["X"].to_list(), 
        ideal_df[f"Y{i+1}"].to_list(), 
        size=5, 
        color="#123456", 
        alpha=0.8
    )

    ideal_dp[-1].append(fig)

for item in ideal_dp: 
    show(row(*item), notebook_handle=True)

## Result Plots 

In [19]:
# Best Ideal functions: [8, 14, 24, 35]

In [20]:
test_df.head()

,x,y,delta_y,number
0,-16.5,-8983.502000,100.000000,24
1,-9.4,-1.698152,100.000000,35
2,-18.5,-12662.127000,100.000000,24
3,1.0,-0.116329,100.000000,14
4,16.0,-2.352354,99.999999,8


In [21]:
# Plot Test Dataframe
fig = figure(width=750, height=750, background_fill_color="#FFFFFF")
fig.circle(
    test_df["x"].to_list(), 
    test_df["y"].to_list(), 
    size=8, 
    color="#123456", 
    alpha=0.8
)
y_ideals = []
for i, row in enumerate(test_df.iterrows()):
    y_ideal = ideal_df[ideal_df["X"] == row[1]["x"]][f"Y{int(row[1]['number'])}"]
    y_ideals.append(float(y_ideal))

parsed_test_df = pd.concat([test_df, pd.Series(y_ideals, name="y_ideal")], axis=1)

fig.triangle(
    parsed_test_df["x"].to_list(), 
    parsed_test_df["y_ideal"].to_list(), 
    size=8, 
    color="red", 
    alpha=0.8
)

GlyphRenderer(id='p5688', ...)

In [22]:
show(fig)
# The red triangles show the y-values of the ideal functions that have been mapped, the grey-blue circles show the original y values of the test method. Please note that some test coordinates are missing, which are the ones for where no ideal function fulfilled the constraint.
# Constraint: 
# ** The criterion for mapping the individual test case to the four ideal functions is that the existing maximum deviation of the calculated regression does not exceed the largest deviation between training dataset (A) and the ideal function (C) chosen for it by more than factor sqrt(2)